In [31]:
import pandas as pd
import folium
import numpy as np
import json
import zipfile
import os
import geopandas as gpd
import time
import datetime
import itertools
import requests

In [2]:
countries = ['thailand', 'cambodia', 'laos', 'malaysia']

## UNZIP DATA

In [ ]:
for c in countries:
    path_to_zip_file = f'data/{c}.zip'
    directory_to_extract_to = 'data/'
    zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
    zip_ref.extractall(directory_to_extract_to)
    zip_ref.close()
    os.unlink(path_to_zip_file)

## READ JSON DATA

In [3]:
def json_reader(path):
    with open(path, 'r', encoding='utf-8') as fh:
        return json.loads(fh.read())

In [4]:
geojsons = {f: json_reader(f"data/{f.capitalize()}_AL4.GeoJson") for f in countries}

## GEOPANDAS FOR ADDITIONAL DATA

In [5]:
# concatenate data with country flags
for c in countries:
    gdf_ = gpd.GeoDataFrame.from_features(geojsons[c])
    gdf_['country'] = c
    try:
        gdf = gdf.append(gdf_, sort = False)
    except:
        gdf = gdf_

# drop some columns and calculate centroids
gdf = gdf[['geometry', 'name', 'localname', 'wikipedia']]
gdf['centr_lat'] = gdf.geometry.centroid.x
gdf['centr_lon'] = gdf.geometry.centroid.y
gdf.head()

nemappend
append
append
append


,geometry,name,localname,wikipedia,centr_lat,centr_lon
0,"(POLYGON ((104.4190919 15.6978084, 104.4190978...",Amnat Charoen Province,จังหวัดอำนาจเจริญ,en:Amnat Charoen Province,104.742211,15.896505
1,"(POLYGON ((100.1949768 14.5658321, 100.1960907...",Ang Thong Province,จังหวัดอ่างทอง,en:Ang Thong Province,100.355477,14.623284
2,"(POLYGON ((100.3278772 13.8041844, 100.3318503...",Bangkok,กรุงเทพมหานคร,en:Bangkok,100.623651,13.771980
3,"(POLYGON ((103.24334 18.357593, 103.2434 18.35...",Bueng Kan Province,จังหวัดบึงกาฬ,en:Bueng Kan Province,103.712064,18.148721
4,"(POLYGON ((102.4355316 14.8494415, 102.4369202...",Buri Ram Province,จังหวัดบุรีรัมย์,en:Buriram Province,102.962893,14.815251


In [6]:
gdf.shape

(136, 6)

## API CALL FOR WEATHER DATA

In [18]:
def convert_to_timestamp(s):
    return time.mktime(datetime.datetime.strptime(s, "%d/%m/%Y").timetuple())

In [9]:
coordinates = [(lat, lon) for lat, lon in zip(gdf.centr_lat.tolist(), gdf.centr_lon.tolist())]
coordinates[0:10]

[(104.74221061528203, 15.896504764601259),
 (100.35547709400848, 14.623284326491325),
 (100.62365087151149, 13.771980174746414),
 (103.71206364706076, 18.148720653792953),
 (102.96289283806522, 14.815251418301179),
 (101.45149021849265, 13.603437278615134),
 (100.02435585047859, 15.13136828922321),
 (101.80252807832218, 16.00790158529956),
 (102.12941835510354, 12.86118366277432),
 (98.7340673664231, 18.78770721827598)]

In [16]:
start_dates = [f'01/{str(ho)}/2018' for ho in range(1, 13)]
date_tuples = [(convert_to_timestamp(start_dates[i]), convert_to_timestamp(start_dates[i+1])) 
                for i in range(len(start_dates)-1)]

In [17]:
date_tuples

[(1514761200.0, 1517439600.0),
 (1517439600.0, 1519858800.0),
 (1519858800.0, 1522533600.0),
 (1522533600.0, 1525125600.0),
 (1525125600.0, 1527804000.0),
 (1527804000.0, 1530396000.0),
 (1530396000.0, 1533074400.0),
 (1533074400.0, 1535752800.0),
 (1535752800.0, 1538344800.0),
 (1538344800.0, 1541026800.0),
 (1541026800.0, 1543618800.0)]

In [22]:
api_call_set = list(itertools.product(date_tuples, coordinates))

hint: http://history.openweathermap.org/data/2.5/history/city?lat=41.85&lon=-87.65

In [33]:
base_url = 'http://history.openweathermap.org/data/2.5/history/city?'

In [36]:
r = requests.get(f"{base_url}lat={api_call_set[0][0][0]}&lon={api_call_set[0][0][1]}&start={api_call_set[0][1][0]}&end={api_call_set[0][1][1]}&APPID=1807857692fc0fa3162fdf680fb8e5fc")

In [37]:
r.json()

{'cod': 401,
 'message': 'Invalid API key. Please see http://openweathermap.org/faq#error401 for more info.'}

## CREATE MAP

In [27]:
m = folium.Map(location = (15.896505, 104.742211))

In [28]:
for k, v in geojsons.items():
    folium.GeoJson(
        v,
        name=k.capitalize()
    ).add_to(m)
    
folium.LayerControl().add_to(m)

In [29]:
m.save('outputs/map.html')